In [1]:
import time
from selenium import webdriver
from selenium.webdriver import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.firefox.service import Service as FirefoxService
from selenium.webdriver.firefox.options import Options as FirefoxOptions

from webdriver_manager.firefox import GeckoDriverManager
from datetime import date
import pandas as pd
import os

In [2]:
outputpath = 'C:/Users/dimut/Box/Home PC/Data Files/glassdoor/reviews/dec2023'

urls = pd.read_csv("C:/Users/dimut/Box/Home PC/Data Files/tech_cos_with_glassdoor_link.csv")
urls = urls[urls['glassdoor_url'].str.len() > 1]
len(urls)

568

In [4]:
driver = None

def initialize_driver():
    global driver

    # Check if the driver is already initialized
    if driver is None:
        # Initialize FirefoxOptions
        firefox_options = FirefoxOptions()
        firefox_options.binary_location = 'C:/Users/dimut/AppData/Local/Mozilla Firefox/firefox.exe'

        # Initialize webdriver with FirefoxOptions and GeckoDriverManager
        driver = webdriver.Firefox(service=Service(GeckoDriverManager().install()), options=firefox_options)

def login_to_glassdoor(email, password):
    global driver

    # Wait for 5 seconds
    time.sleep(5)

    # Navigate to the login page
    driver.get("https://www.glassdoor.com/profile/login_input.htm")

    # Wait for 3 seconds
    time.sleep(3)

    # Enter email and press Enter
    email_input = driver.find_element(By.ID, 'inlineUserEmail')
    email_input.send_keys(email)
    email_input.send_keys(Keys.ENTER)

    # Wait for 3 seconds
    time.sleep(3)

    # Enter password and press Enter
    password_input = driver.find_element(By.ID, 'inlineUserPassword')
    password_input.send_keys(password)
    password_input.send_keys(Keys.ENTER)

In [23]:
driver = None
initialize_driver()
time.sleep(3)
login_to_glassdoor('sumana.gamage23@gmail.com', 'cubs123##')

In [29]:
company_counter = 0
for index, row in urls.iterrows():

    company_counter = company_counter + 1
    
    
    if company_counter<339:
        continue
        
    if company_counter>400:
        break

    print(company_counter)
    
    # if company_counter % 50 == 0:
    #     cont = input("continue? (y/n) ")
    #
    #     if cont == "n":
    #         break

    ticker = row['Symbol']
    url = row['glassdoor_url']

    print(ticker)
    driver.get(url)
    time.sleep(3)

    try:
        overallReviewCount = int(driver.find_element(By.CSS_SELECTOR, '[data-test="overallReviewCount"]').find_elements(By.TAG_NAME,'strong')[0].text.replace(",", ""))
        print(overallReviewCount)
    except:
        print("trying again")
        driver.close()
        time.sleep(50)
        driver = None
        time.sleep(3)
        initialize_driver()
        time.sleep(3)
        login_to_glassdoor('sumana.gamage23@gmail.com', 'cubs123##')
        continue

    if overallReviewCount < 50:
        print('overallReviewCount < 50')
        continue

    f = open(outputpath + '/' + ticker + '.txt', 'a', encoding='utf-8')

    page_nos = range(3, round(overallReviewCount / 10) - 1, max(round(overallReviewCount / 180), 2))

    p = 0
    for pno in page_nos:
        
        # print(pno)
        p=p+1
        
        # if p>3:
        #     break

        temp_url = url[0:(len(url) - 4)] + "_P" + str(pno+1) + ".htm?sort.sortType=RD"
        # print(temp_url)
        try:
            driver.get(temp_url)
        except:
            try:
                time.sleep(5)
                driver.get(temp_url)
            except:
                print(temp_url + " failed")
                break
        time.sleep(3)
        
        try:
            email_input2 = driver.find_element(By.ID,"hardsellUserEmail")
            email_input2.send_keys('sumana.gamage23@gmail.com')
            email_input2.send_keys(Keys.ENTER)
            
            time.sleep(3)
            
            pw_input = driver.find_element(By.ID,"hardsellUserPassword")
            pw_input.send_keys('cubs123##')
            pw_input.send_keys(Keys.ENTER)
        except:
            pass

        try:
            gdReview = driver.find_elements(By.CLASS_NAME, "review-details__review-details-module__topReview")
            # print(len(gdReview))
            if len(gdReview) == 0:
                break

            time.sleep(3)

            for gdr in gdReview:
                    time.sleep(1)
                # try:
                    title = gdr.find_element(By.TAG_NAME, 'h2').text
                    rating = gdr.find_element(By.CLASS_NAME, "review-details__review-details-module__overallRating").text
                    current_employee = gdr.find_element(By.CLASS_NAME, "review-details__review-details-module__employeeDetails").text
                    review_date = gdr.find_element(By.CLASS_NAME, "review-details__review-details-module__reviewDate").text # new
                    job_title = gdr.find_element(By.CLASS_NAME, "review-details__review-details-module__employee").text  # new
                    
                    try: # new
                        gdr.find_element(By.CLASS_NAME, "expand-button__expand-button-module__ExpandButton").click()
                        pros =  gdr.find_element(By.CSS_SELECTOR, 'span[data-test="pros"]').text.replace('\n', ' ').replace('|',' ')
                        cons = gdr.find_element(By.CSS_SELECTOR, 'span[data-test="cons"]').text.replace('\n', ' ').replace('|',' ')
                    except:
                        pros =  gdr.find_element(By.CSS_SELECTOR, 'span[data-test="pros"]').text.replace('\n', ' ').replace('|',' ')
                        cons = gdr.find_element(By.CSS_SELECTOR, 'span[data-test="cons"]').text.replace('\n', ' ').replace('|',' ')
                    
                    recommend = ''
                    ceoapproval=''
                    businessoutlook=''    
                    review_div = gdr.find_element(By.CLASS_NAME, 'review-details__review-details-module__iconContainer')
                    svg_elements = review_div.find_elements(By.TAG_NAME, 'svg')
                    if len(svg_elements)==3:
                        for svg in svg_elements:
                            recommend = svg_elements[0].get_attribute('innerHTML')
                            ceoapproval = svg_elements[1].get_attribute('innerHTML')
                            businessoutlook = svg_elements[2].get_attribute('innerHTML')
  
                    recommend = recommend[:70]
                    ceoapproval = ceoapproval[:70]
                    businessoutlook = businessoutlook[:70]
                  
                    data_line = str(pno) + "|" + ticker + "|" + title + "|" + rating + "|" + current_employee + "|"+review_date+"|"+job_title+"|"+pros+"|"+recommend+"|"+ceoapproval+"|"+businessoutlook
                    data_line = data_line.replace('\n', ' ')
                    f.write(data_line+"\n")
        except:
            try:
                print("trying again")
                driver.close()
                time.sleep(50)
                driver = None
                time.sleep(3)
                initialize_driver()
                time.sleep(3)
                login_to_glassdoor('sumana.gamage23@gmail.com', 'cubs123##')
                time.sleep(60)
            except:
                break

    f.close()
    time.sleep(30)
# driver.close()

339
LQDT
238
340
MYPS
144
341
AMSWA
14
overallReviewCount < 50
342
SCWX
601
343
DDI
77
344
MARA
3
overallReviewCount < 50
345
MITK
72
346
SMRT
88
347
LASR
72
348
PRTH
23
overallReviewCount < 50
349
SSTI
46
overallReviewCount < 50
350
MVIS
35
overallReviewCount < 50
351
CPSI
207
352
WISH
trying again
353
ONTF
355
trying again
354
LAW
238
355
DMRC
31
overallReviewCount < 50
356
XPER
177
357
PETS
68
358
TSP
320
359
SCPL
107
360
SKYT
40
overallReviewCount < 50
361
PRTS
214
362
SFIX
2412
363
EVER
229
364
CMTL
144
365
RMNI
293
366
CYXT
135
367
TLS
147
368
UIS
4256
369
TGAN
177
370
AEVA
177
371
HIPO
5
overallReviewCount < 50
372
VLD
36
overallReviewCount < 50
373
BLND
216
374
WEAV
trying again
375
NN
31
overallReviewCount < 50
376
LVOX
125
trying again
377
EGAN
432
378
ARLO
95
379
SKLZ
305
380
BARK
279
381
VUZI
8
overallReviewCount < 50
382
TREE
283
383
RBT
177
384
VERI
140
385
SOUN
111
386
UPLD
321
387
BCOV
270
388
ABSI
41
overallReviewCount < 50
389
KLTR
234
390
PEAR
49
overallReviewCount <

In [8]:
firefox_options = FirefoxOptions()
firefox_options.binary_location = 'C:/Users/dimut/AppData/Local/Mozilla Firefox/firefox.exe'
driver = webdriver.Firefox(service=Service(GeckoDriverManager().install()), options=firefox_options)

time.sleep(5)
driver.get("https://www.glassdoor.com/profile/login_input.htm")

time.sleep(3)
driver.find_element(By.ID, 'inlineUserEmail').send_keys('sumana.gamage23@gmail.com')
driver.find_element(By.ID, 'inlineUserEmail').send_keys(Keys.ENTER)

time.sleep(3)
driver.find_element(By.ID, 'inlineUserPassword').send_keys('cubs123##')
driver.find_element(By.ID, 'inlineUserPassword').send_keys(Keys.ENTER)

132

In [28]:
email_input2 = driver.find_element(By.ID,"hardsellUserEmail")
email_input2.send_keys('sumana.gamage23@gmail.com')
email_input2.send_keys(Keys.ENTER)

time.sleep(3)

pw_input = driver.find_element(By.ID,"hardsellUserPassword")
pw_input.send_keys('cubs123##')
pw_input.send_keys(Keys.ENTER)

In [29]:
# #test
# row = urls.iloc[2]
# ticker = row['Symbol']
# url = row['glassdoor_url']
# 
# print(ticker)
# driver.get(url)
# time.sleep(3)
# 
# overallReviewCount = int(
#             driver.find_element(By.CSS_SELECTOR, '[data-test="overallReviewCount"]').find_elements(By.TAG_NAME,'strong')[0].text.replace(",", ""))
# 
# page_nos = range(1, round(overallReviewCount / 10) - 1, max(round(overallReviewCount / 200), 2))
# 
# pno = page_nos[4]
# 
# temp_url = url[0:(len(url) - 4)] + "_P" + str(pno+1) + ".htm?sort.sortType=RD"
# driver.get(temp_url)
# 
# # test
# gdReview = driver.find_elements(By.CLASS_NAME, "review-details__review-details-module__topReview")
# len(gdReview)
# gdr = gdReview[5]
# gdr.find_element(By.TAG_NAME, 'h2').text
# gdr.find_element(By.CLASS_NAME, "review-details__review-details-module__overallRating").text
# gdr.find_element(By.CLASS_NAME, "review-details__review-details-module__employeeDetails").text
# 
# #test
# try:
#     gdr.find_element(By.CLASS_NAME, "expand-button__expand-button-module__ExpandButton").click()
#     pros =  gdr.find_element(By.CSS_SELECTOR, 'span[data-test="pros"]').text.replace('\n', ' ')
#     cons = gdr.find_element(By.CSS_SELECTOR, 'span[data-test="cons"]').text.replace('\n', ' ')
# except:
#     pros =  gdr.find_element(By.CSS_SELECTOR, 'span[data-test="pros"]').text.replace('\n', ' ')
#     cons = gdr.find_element(By.CSS_SELECTOR, 'span[data-test="cons"]').text.replace('\n', ' ')
#     
# #test
# pros
# 
# review_div = gdr.find_element(By.CLASS_NAME, 'review-details__review-details-module__iconContainer')
# 
# # Find all SVG elements within the div
# svg_elements = review_div.find_elements(By.TAG_NAME, 'svg')
# for svg in svg_elements:
#     recommend = svg_elements[0].get_attribute('innerHTML')
#     ceoapproval = svg_elements[1].get_attribute('innerHTML')
#     businessoutlook = svg_elements[2].get_attribute('innerHTML')
#     
# gdr.find_element(By.CLASS_NAME, 'review-details__review-details-module__iconContainer').text


GOOG


In [9]:
# file_list = os.listdir(outputpath)
# file_list = [x.split('.')[0] for x in file_list]
# 
# urls = urls.loc[~urls['Symbol'].str.contains('|'.join(file_list))]
# # urls = urls.iloc[0:100]
# print(len(urls.index))

0


In [ ]:
company_counter = 0
for index, row in urls.iterrows():

    company_counter = company_counter + 1
    print(company_counter)
    
    if company_counter<400:
        continue
        
    if company_counter>402:
        break

    # if company_counter % 50 == 0:
    #     cont = input("continue? (y/n) ")
    #
    #     if cont == "n":
    #         break

    ticker = row['Symbol']
    url = row['glassdoor_url']

    print(ticker)
    driver.get(url)
    time.sleep(3)

    try:
        overallReviewCount = int(driver.find_element(By.CSS_SELECTOR, '[data-test="overallReviewCount"]').find_elements(By.TAG_NAME,'strong')[0].text.replace(",", ""))
        print(overallReviewCount)
    except:
        print('overallReviewCount int')
        continue

    if overallReviewCount < 50:
        print('overallReviewCount < 50')
        continue

    f = open(outputpath + '/' + ticker + '.txt', 'a', encoding='utf-8')

    page_nos = range(3, round(overallReviewCount / 10) - 1, max(round(overallReviewCount / 180), 2))

    p = 0
    for pno in page_nos:
        # print(pno)
        p=p+1
        
        # if p>3:
        #     break

        temp_url = url[0:(len(url) - 4)] + "_P" + str(pno+1) + ".htm?sort.sortType=RD"
        # print(temp_url)
        try:
            driver.get(temp_url)
        except:
            try:
                time.sleep(5)
                driver.get(temp_url)
            except:
                print(temp_url + " failed")
                break
        time.sleep(3)

        try:
            gdReview = driver.find_elements(By.CLASS_NAME, "review-details__review-details-module__topReview")
            # print(len(gdReview))
            if len(gdReview) == 0:
                break

            time.sleep(3)

            for gdr in gdReview:
                    time.sleep(1)
                # try:
                    title = gdr.find_element(By.TAG_NAME, 'h2').text
                    rating = gdr.find_element(By.CLASS_NAME, "review-details__review-details-module__overallRating").text
                    current_employee = gdr.find_element(By.CLASS_NAME, "review-details__review-details-module__employeeDetails").text
                    review_date = gdr.find_element(By.CLASS_NAME, "review-details__review-details-module__reviewDate").text # new
                    job_title = gdr.find_element(By.CLASS_NAME, "review-details__review-details-module__employee").text  # new
                    
                    try: # new
                        gdr.find_element(By.CLASS_NAME, "expand-button__expand-button-module__ExpandButton").click()
                        pros =  gdr.find_element(By.CSS_SELECTOR, 'span[data-test="pros"]').text.replace('\n', ' ').replace('|',' ')
                        cons = gdr.find_element(By.CSS_SELECTOR, 'span[data-test="cons"]').text.replace('\n', ' ').replace('|',' ')
                    except:
                        pros =  gdr.find_element(By.CSS_SELECTOR, 'span[data-test="pros"]').text.replace('\n', ' ').replace('|',' ')
                        cons = gdr.find_element(By.CSS_SELECTOR, 'span[data-test="cons"]').text.replace('\n', ' ').replace('|',' ')
                    
                    recommend = ''
                    ceoapproval=''
                    businessoutlook=''    
                    review_div = gdr.find_element(By.CLASS_NAME, 'review-details__review-details-module__iconContainer')
                    svg_elements = review_div.find_elements(By.TAG_NAME, 'svg')
                    if len(svg_elements)==3:
                        for svg in svg_elements:
                            recommend = svg_elements[0].get_attribute('innerHTML')
                            ceoapproval = svg_elements[1].get_attribute('innerHTML')
                            businessoutlook = svg_elements[2].get_attribute('innerHTML')
  
                    recommend = recommend[:70]
                    ceoapproval = ceoapproval[:70]
                    businessoutlook = businessoutlook[:70]
                  
                    data_line = str(pno) + "|" + ticker + "|" + title + "|" + rating + "|" + current_employee + "|"+review_date+"|"+job_title+"|"+pros+"|"+recommend+"|"+ceoapproval+"|"+businessoutlook
                    data_line = data_line.replace('\n', ' ')
                    f.write(data_line+"\n")
                    # f.write(
                    #     str(pno) + "|-|" + ticker + "|-|" + title + "|-|" + rating + "|-|" + current_employee + "|-|"
                    #     + date_jobtitle + "|-|" + review_body_text.replace("\n",
                    #                                                        "---") + "|-|" + review1 + "|-|" + review2
                    #     + "|-|" + review3 + "|-|" + pros + "|-|" + cons + "|-|" + work_life_bal + "|-|" + culture_value
                    #     + "|-|" + diversity + "|-|" + carrier + "|-|" + compensation + "|-|" + management + "\n")
                    # 
                # except Exception as e:
                #     time.sleep(10)
                #     print("skip")
                #     # nextButton = driver.find_elements(By.CLASS_NAME, 'nextButton')
                #     # if len(nextButton) == 1:
                #     #     if nextButton[0].is_enabled():
                #     #         page_no = page_no + 1
                #     #         nextButton[0].click()
                #     #         time.sleep(5)
                #     break


        except:
            break

    f.close()
    time.sleep(30)
# driver.close()


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [ ]:
  review_body_text = gdr.find_element(By.CLASS_NAME, "reviewBodyCell ").text
                    reviewBody = gdr.find_element(By.CLASS_NAME, "reviewBodyCell ").find_elements(By.CLASS_NAME,
                                                                                                  "SVGInline-svg")

                    if len(reviewBody) == 3:
                        try:
                            review1 = reviewBody[0].find_element(By.TAG_NAME, "path").get_attribute("d")[0:20]
                        except:
                            if len(reviewBody[0].find_elements(By.TAG_NAME, "circle")) == 1:
                                review1 = "circle"
                            elif len(reviewBody[0].find_elements(By.TAG_NAME, "rect")) == 1:
                                review1 = "rect"
                            else:
                                review1 = ""
                        try:
                            review2 = reviewBody[1].find_element(By.TAG_NAME, "path").get_attribute("d")[0:20]
                        except:
                            if len(reviewBody[1].find_elements(By.TAG_NAME, "circle")) == 1:
                                review2 = "circle"
                            elif len(reviewBody[1].find_elements(By.TAG_NAME, "rect")) == 1:
                                review2 = "rect"
                            else:
                                review2 = ""
                        try:
                            review3 = reviewBody[2].find_element(By.TAG_NAME, "path").get_attribute("d")[0:20]
                        except:
                            if len(reviewBody[2].find_elements(By.TAG_NAME, "circle")) == 1:
                                review3 = "circle"
                            elif len(reviewBody[2].find_elements(By.TAG_NAME, "rect")) == 1:
                                review3 = "rect"
                            else:
                                review3 = ""

                        pros = gdr.find_element(By.CSS_SELECTOR, '[data-test="pros"]').text
                        cons = gdr.find_element(By.CSS_SELECTOR, '[data-test="cons"]').text

                        work_life_bal = ""
                        culture_value = ""
                        diversity = ""
                        carrier = ""
                        compensation = ""
                        management = ""

                        if len(gdr.find_elements(By.CLASS_NAME, 'tooltipContainer')) == 1 and len(
                                gdr.find_element(By.CLASS_NAME, 'tooltipContainer').find_elements(By.TAG_NAME,
                                                                                                  "li")) == 6:
                            lis = gdr.find_element(By.CLASS_NAME, 'tooltipContainer').find_elements(By.TAG_NAME, "li")
                            li_count = 1
                            for li in lis:
                                if li_count == 1:
                                    work_life_bal = li.find_elements(By.TAG_NAME, "div")[1].get_attribute('class')
                                if li_count == 2:
                                    culture_value = li.find_elements(By.TAG_NAME, "div")[1].get_attribute('class')
                                if li_count == 3:
                                    diversity = li.find_elements(By.TAG_NAME, "div")[1].get_attribute('class')
                                if li_count == 4:
                                    carrier = li.find_elements(By.TAG_NAME, "div")[1].get_attribute('class')
                                if li_count == 5:
                                    compensation = li.find_elements(By.TAG_NAME, "div")[1].get_attribute('class')
                                if li_count == 6:
                                    management = li.find_elements(By.TAG_NAME, "div")[1].get_attribute('class')
                                # css-1nuumx7 e1hd5jg10 4 stars,s88v13 5 stars, 18v8tui 2, vl2edp 3, xd4dom 1
                                li_count = li_count + 1
